In [23]:
# get data
import pandas as pd
# ML stuff
from mne.decoding import CSP
from mne_bids import BIDSPath, read_raw_bids
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline
# series to series transformer
from sktime.transformations.series.detrend import Detrender
from sktime.forecasting.trend import TrendForecaster

# panel to panel transformers
from sktime_neuro.transformations.panel.baselinecorrection import \
    BaselineCorrectionTransformer as Baseline
from sktime_neuro.transformations.series.filterforseries import FilterforSeries
from sktime_neuro.transformations.series.seriesdownsampling import \
    SeriesDownsampling
from sktime_neuro.transformations.series_to_panel.eeg_epoching import epoch
# epoching
from sktime_neuro.utils import mne_processing as utils
# To get data paths
from pooch import os_cache

## Get Bids Data into sktime Series format

In [24]:
# example can be downloaded here: https://osf.io/cj2dr/
# Get pooch cahe dir, since our data is there by convention now.
root = os_cache("sktime_neuro") / "eeg_matchingpennies"
bids_path = BIDSPath(
    subject="08", task="matchingpennies", suffix="eeg", datatype="eeg", root=root
)

raw = read_raw_bids(bids_path=bids_path, verbose=False)

# mne assumes shape channels*timepoints;sktime assumes shape timepoints*channels
data = raw.get_data().transpose()

/tmp/ipykernel_71188/3419896517.py:8: RuntimeWarning: Omitted 300 annotation(s) that were outside data range.
  raw = read_raw_bids(bids_path=bids_path, verbose=False)
/tmp/ipykernel_71188/3419896517.py:8: RuntimeWarning: Omitted 300 annotation(s) that were outside data range.
  raw = read_raw_bids(bids_path=bids_path, verbose=False)
/tmp/ipykernel_71188/3419896517.py:8: RuntimeWarning: Did not find any eeg.json associated with sub-08_task-matchingpennies.

The search_str was "/home/patchouli/.cache/sktime_neuro/eeg_matchingpennies/sub-08/**/eeg/sub-08*eeg.json"
  raw = read_raw_bids(bids_path=bids_path, verbose=False)


## Apply Series to Series Transformer

In [25]:
downsampled_data = SeriesDownsampling(2).fit_transform(data)

In [26]:
#detrended_data = Detrender().fit_transform(pd.DataFrame(downsampled_data))
detrended_data = downsampled_data

In [27]:
filtered_data = FilterforSeries(l_freq=8, h_freq=16, sfreq=250).fit_transform(
    detrended_data
)

Setting up band-pass filter from 8 - 16 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 8.00
- Lower transition bandwidth: 2.00 Hz (-6 dB cutoff frequency: 7.00 Hz)
- Upper passband edge: 16.00 Hz
- Upper transition bandwidth: 4.00 Hz (-6 dB cutoff frequency: 18.00 Hz)
- Filter length: 413 samples (1.652 sec)



/home/patchouli/programming/foss/sktime-neuro/sktime_neuro/transformations/series/filterforseries.py:87: RuntimeWarning: filter_length (413) is longer than the signal (1), distortion is likely. Reduce filter length or filter a longer signal.
  z = filter.filter_data(


## Epoch

In [28]:
annotation = utils.create_annotation(raw)

data, labels = epoch(
    Z=filtered_data,
    annotation=annotation,
    labels=["right/2", "left/1"],
    interval=[-0.5, 1.5],
    sfreq=250,
)

## Apply Panel-To-Panel Transformers

In [29]:
bl_corrected = Baseline(upper=0.5, fs=250).fit_transform(data)

ValueError: Found array with: 0 instance(s) but a minimum of: 1 is required.

## Classify Trials

In [ ]:
# Assemble a classifier

# filterer = PaneltoPanelFilter() # should not be done for short samples
csp = CSP(transform_into="average_power")
lda = LinearDiscriminantAnalysis()


param_grid = {
    "CSP__n_components": [1, 2, 3, 4, 5, 7, 10],
}

# Use scikit-learn Pipeline with cross_val_score function
pipe = Pipeline(steps=[("CSP", csp), ("LDA", lda)])

search = GridSearchCV(pipe, param_grid, n_jobs=-1)
search.fit(bl_corrected, labels)
print("Best parameter (CV score=%0.3f):" % search.best_score_)
print(search.best_params_)